In [7]:
import pandas as pd
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

In [8]:
# Load the test CSV
test_csv_path = (
    "/Users/richmondsin/Desktop/DSA4264/DSA4264-Detoxify/model-1/distilbert/test.csv"
)
df = pd.read_csv(test_csv_path)

In [9]:
# Set the column names
text_column = "text"  # Column containing the text data
true_labels_column = "Classification"  # Column containing the true labels

# Ensure data is on GPU if available
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Define model paths
model_paths = [
    "/Users/richmondsin/Desktop/DSA4264/DSA4264-Detoxify/model-1/distilbert/model/multilingual_distilbert_model.pth",  # Replace with actual path for model 1
    "/Users/richmondsin/Desktop/DSA4264/DSA4264-Detoxify/model-1/distilbert/model/multilingual_distilbert_model_5k.pth",  # Replace with actual path for model 2
    "/Users/richmondsin/Desktop/DSA4264/DSA4264-Detoxify/model-1/distilbert/model/multilingual_distilbert_model_15k.pth",  # Replace with actual path for model 3
]

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

In [10]:
# Function for predicting text with tqdm progress bar


def predict_texts(texts, model, tokenizer, device):
    model.eval()
    predicted_classes = []

    # Use tqdm to show progress bar
    for text in tqdm(texts, desc="Predicting", unit="text"):
        inputs = tokenizer(
            text, return_tensors="pt", truncation=True, padding=True, max_length=128
        ).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1).item()
        predicted_classes.append(predicted_class)

    return predicted_classes


# Define class labels
class_labels = [
    "No Hate/Toxic",
    "Toxic 1",
    "Toxic 2",
    "Toxic 3",
    "Hate 1",
    "Hate 2",
    "Hate 3",
]

# Evaluate with each model and print the classification report
for i, model_path in enumerate(model_paths, start=1):
    print(f"Evaluating Model {i}...")

    # Load the model
    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-multilingual-cased", num_labels=7
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)

    # Predict labels
    df[f"predicted_labels_model_{i}"] = predict_texts(
        df[text_column].tolist(), model, tokenizer, device
    )

    # Generate and print classification report
    print(f"\nClassification Report for Model {i}:\n")
    print(
        classification_report(
            df[true_labels_column],
            df[f"predicted_labels_model_{i}"],
            target_names=class_labels,
        )
    )

Evaluating Model 1...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/dj/ph12jwsx1tlgrcwbvx_f6dzw0000gn/T/ipykernel_7735/3432904603.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loade


Classification Report for Model 1:

               precision    recall  f1-score   support

No Hate/Toxic       0.45      0.45      0.45      2250
      Toxic 1       0.20      0.22      0.21      2250
      Toxic 2       0.24      0.57      0.34      2250
      Toxic 3       0.81      0.04      0.08      2250
       Hate 1       0.47      0.38      0.42      2250
       Hate 2       0.54      0.67      0.60      2250
       Hate 3       0.98      0.40      0.57      2250

     accuracy                           0.39     15750
    macro avg       0.53      0.39      0.38     15750
 weighted avg       0.53      0.39      0.38     15750

Evaluating Model 2...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/dj/ph12jwsx1tlgrcwbvx_f6dzw0000gn/T/ipykernel_7735/3432904603.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loade


Classification Report for Model 2:

               precision    recall  f1-score   support

No Hate/Toxic       0.57      0.70      0.63      2250
      Toxic 1       0.39      0.33      0.36      2250
      Toxic 2       0.40      0.38      0.39      2250
      Toxic 3       0.99      0.98      0.98      2250
       Hate 1       0.77      0.78      0.77      2250
       Hate 2       0.94      0.94      0.94      2250
       Hate 3       0.99      0.99      0.99      2250

     accuracy                           0.73     15750
    macro avg       0.72      0.73      0.72     15750
 weighted avg       0.72      0.73      0.72     15750

Evaluating Model 3...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/dj/ph12jwsx1tlgrcwbvx_f6dzw0000gn/T/ipykernel_7735/3432904603.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loade


Classification Report for Model 3:

               precision    recall  f1-score   support

No Hate/Toxic       0.73      0.70      0.72      2250
      Toxic 1       0.56      0.68      0.61      2250
      Toxic 2       0.88      0.71      0.79      2250
      Toxic 3       0.99      0.98      0.98      2250
       Hate 1       0.76      0.83      0.79      2250
       Hate 2       1.00      0.94      0.97      2250
       Hate 3       0.99      0.99      0.99      2250

     accuracy                           0.83     15750
    macro avg       0.84      0.83      0.84     15750
 weighted avg       0.84      0.83      0.84     15750

